In [28]:
#ODE_Fit version that uses the IPNewton() optimization algorithm from Optim.jl
using Pkg, Plots, XLSX, Dates, DataFrames, Random, Statistics
using NBInclude
using DifferentialEquations
using ForwardDiff, Optim, Sundials
@nbinclude("ODE_models.ipynb")
@nbinclude("Norm_functions.ipynb")
@nbinclude("HelperFunctions.ipynb")

data_to_df (generic function with 1 method)

In [29]:
function SecondOrderFit(;ODE_model, ODE_vars, data_obs, t_obs, t_all, x0, x_LBs, x_UBs, UBs,
                         param_opt_indices, param_fix_indices, IC_opt_indices, IC_fix_indices,
                         params_fix, ICs_fix, f_calc_ICs, N0, norm, norm_weights, norm_scale,
                         optimizer, make_plots::Bool, integrator_options::Dict,
                         optim_options, constraints, autodiff, convergence_report::Bool = false)
    
    #THE ARGUMENTS
    #ODE_model: The function that computes the vector dY/dt given a vector Y, the time t, and parameter values. 
    #ODE_vars: The ODE variable names, given as an array of Strings
    #data_obs: A DataFrame of observed data values, where each column corresponds to an ODE variable.
    #t_obs: An array of the time values at which the data in data_obs were observed.  
    #t_all: An array of time values for which to plot the predicted value 
    #x0: The initial guess (Array of Reals) for the optimizer     
    #x_LBs: The lower bounds for all of the variables being fitted ([param_LBs; IC_LBs])
    #x_UBs: The upper bounds for all the variables being fitted (should be [1,1,1,...,l])
    #UBs: The original (unscaled) upper bounds on all the parameters & ICs being fitted 
    
    #optimizer: one of IPNewton(), ConjugateGradient(), or GradientDescent()
    #optim_options: An object of the form Optim.Options(xtol = xtol_abs, gtol = gtol_rel,...)
    #constraints: A TwiceDifferentiableConstraints object 
    #autodiff: either :finite or :forward 
    #convergence_report: Whether or not to display the convergence report after the optimization finishes.
    
    ########################################################################
    #Step 1: Get some info needed in order to define the objective function
    ########################################################################
    
    #Scale the observed data 
    data_obs_scaled = data_obs ./ N0
    
    vars_to_fit = names(data_obs)    
    num_params_opt = length(param_opt_indices)
    num_ICs_opt = length(IC_opt_indices)
    
    num_params = num_params_opt + length(param_fix_indices)
    num_ICs_and_IC_ratios = num_ICs_opt + length(IC_fix_indices)
    
    #Unpack integrator options
    rtol = get(integrator_options, :rtol, 1e-8)
    atol = get(integrator_options, :atol, 1e-8)
    integrator = get(integrator_options, :integrator, Tsit5())
    
    #Get the original upper bounds 
    param_UBs = UBs[1:num_params_opt]
    IC_UBs = UBs[num_params_opt+1:end]
    
    #########################################
    #Step 2: Define the objective function
    #########################################
    
    function objective(x)
        
        #Unpack scaled params & ICs/IC ratios
        params_opt_scaled = x[1:num_params_opt]
        IC_ratios_opt_scaled = x[num_params_opt+1:end]
        
        #Scale the params & IC ratios back
        params_opt = params_opt_scaled .* param_UBs
        IC_ratios_opt = IC_ratios_opt_scaled .* IC_UBs
        
        #Create an array to store the ODE parameters
        ODE_params = Array{Real,1}(undef, num_params) 
        
        #Populate ODE_params with the parameter values
        ODE_params[param_opt_indices] = params_opt
        ODE_params[param_fix_indices] = params_fix
        
        #Create an array to store the ICs and IC ratios
        ICs_and_IC_ratios = Array{Real,1}(undef, num_ICs_and_IC_ratios) 
        
        #Populate ICs_and_IC_ratios 
        ICs_and_IC_ratios[IC_opt_indices] = IC_ratios_opt
        ICs_and_IC_ratios[IC_fix_indices] = ICs_fix
        
        #Calculate the ODE Initial Conditions (for the integrator)
        ODE_ICs = f_calc_ICs(ICs_and_IC_ratios)
        
        #Scale the initial condition
        ODE_ICs_scaled = ODE_ICs ./ N0
        
        #Get the time interval for which to solve the ODE
        t_span = 1.0 .* [t_obs[1], t_obs[end]]
        
        #Compute the numerical solution of the ODE  
        ODE_prob = ODEProblem(ODE_model, ODE_ICs_scaled, t_span, ODE_params)
        ODE_sol = solve(ODE_prob, integrator, reltol = rtol, abstol = atol, saveat = t_obs, dt = 0.01)
        
        #Note: For adaptive step size integrators, the dt argument is just the initial step size.
        #Setting an initial dt seems to help the "Warning initial dt set to NaN" issue)
        
        code = ODE_sol.retcode 
        
        #If the integration was successful, compute the norm (aka loss). 
        if code == :Success
        
            #Convert ODE_sol to a DataFrame (for convenience)
            ODE_sol = DataFrame(ODE_sol', ODE_vars)

            norm_sum = 0.0
            for var in vars_to_fit
                var_pred = ODE_sol[:,var]
                var_obs = data_obs_scaled[:,var]
                norm_sum += norm(var_pred, var_obs; w = norm_weights)
            end
            
            mean_norm = norm_sum / length(vars_to_fit)
            return mean_norm 
           
       #If the integration somehow failed (e.g. due to blow-up), return Inf
       else
            println("Uh oh, the ODE solver failed. The return code is $code")
            println("The x values that were passed to objective(): ", x)
            return Inf
       end
        
    end
    #End of Objective Function definition
    
    
    ###############################
    #STEP 3: Call to the Optimizer
    ###############################
    
    local out::Any       #Gets assigned the Optim result object
    local AD_fail::Bool  #To keep track of any autodifferentiation fails
    
  
    Random.seed!(2021)
    #Attempt to optimize using automatic differentiation (via ForwardDiff).
    try 
        Twice_diff = TwiceDifferentiable(objective, x0, autodiff = :forward)
        out = Optim.optimize(Twice_diff, constraints, x0, IPNewton(), optim_options)
        AD_fail = false
    
    #If the optimization w/AD fails, attempt it again using finite differencing instead.  
    catch    
        println("Optimization with ForwardDiff failed. Switching to finite differencing...", "\n")
        Twice_diff = TwiceDifferentiable(objective, x0, autodiff = :finite)
        out = Optim.optimize(Twice_diff, constraints, x0, IPNewton(), optim_options)
        AD_fail = true
    end    
        
    #The minimizer obtained by the optimizer. This will be an array of Float64's
    x_opt = out.minimizer 
    
    #Displays the convergence report when show_trace is true
    if convergence_report == true
        println(out)
    end

    
    #######################################################################
    #STEP 4: Compute the ODE solution using the minimizer that Optim found 
    #        and plot it against the observed data (if make_plots == true)
    #######################################################################
 
    if make_plots == true
        y = ODE_sol(ODE_model, ODE_vars, x_opt, params_fix, ICs_fix, param_opt_indices, param_fix_indices,
                     IC_opt_indices, IC_fix_indices, param_UBs, IC_UBs, f_calc_ICs, N0, t_all, int_options)
        
        for var in vars_to_fit
           plot(y.t, y[:,var],
                 size = (450,350),
                 line = :line,
                 lw = 1.5,
                 label = "$var (pred)",
                 legend = :topleft,
                 xlabel = "t",
                 ylabel = "Number",
                 title = "$var")
            
           display(plot!(t_obs, data_obs[:,var],
                 line = :scatter,
                 markersize = 2,
                 markerstrokewidth = 0,
                 label = "$var (obs, fit)"))
        end
    end
    
    return (minimizer = out.minimizer, minimum = out.minimum, AD_fail = AD_fail)  
end

SecondOrderFit (generic function with 1 method)

In [26]:
#Note: The norm functions are a bit faster when the loops are written out explicitly, 
#compared with using broadcasting, hence why the loops are written out here. 

function mean_square_error(x,y; w = fill(1.0, length(x)))
    sum = 0.0
    for i=1:length(x)
        sum += w[i]*(x[i] - y[i])^2
    end
    return sum/length(x)
end

function mean_abs_error(x,y; w = fill(1.0, length(x)))
    sum = 0.0
    for i=1:length(x)
        sum += w[i]*abs(x[i] - y[i])
    end
    return sum/length(x)
end

function mean_abs_rel_error(x,y; w = fill(1.0, length(x)))
    sum = 0.0
    for i=1:length(x)
        sum += w[i]*abs(x[i]/y[i] - 1)
    end
    return sum/length(x)
end

function mean_square_rel_error(x,y; w = fill(1.0, length(x)))
    sum = 0.0
    for i=1:length(x)
        sum += w[i]*(x[i]/y[i] - 1)^2
    end
    return sum/length(x)
end

mean_square_rel_error (generic function with 1 method)